In [1]:
#PACKAGES

import pandas as pd
import yfinance as yf
import talib
import numpy as np
import requests
from io import StringIO
from datetime import date, datetime, timedelta
import praw
import emoji
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt

In [2]:
#TICKERS

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0"}
html = requests.get(url, headers=headers).text
sp500 = pd.read_html(StringIO(html))[0]
tickers = sp500['Symbol'].tolist()
ticker_map = {
    "BF.B": "BF-B",
    "BRK.B": "BRK-B"
}
tickers = [ticker_map.get(t, t) for t in tickers] 
tickers = sorted(tickers)

In [3]:
#VARIABLES

def calculate_variables(tickers):
    all_data = []

    for i, ticker in enumerate(tickers, 1):
        try:
            t = yf.Ticker(ticker)

            bs = t.quarterly_balance_sheet.T
            is_ = t.quarterly_income_stmt.T
            cf = t.quarterly_cashflow.T

            bs_cols = ['Total Assets', 'Total Stockholder Equity', 'Total Current Assets',
                       'Total Current Liabilities', 'Inventory', 'Cash', 'Total Debt']
            is_cols = ['Net Income', 'EBITDA', 'EBIT', 'Operating Income',
                       'Gross Profit', 'Total Revenue', 'Interest Expense', 'Research Development']
            cf_cols = ['Total Cash From Operating Activities', 'Capital Expenditures', 'Free Cash Flow']

            bs = bs[[c for c in bs_cols if c in bs.columns]]
            is_ = is_[[c for c in is_cols if c in is_.columns]]
            cf = cf[[c for c in cf_cols if c in cf.columns]]

            df = bs.join(is_, how="outer").join(cf, how="outer")
            df = df.tail(1)

            info = t.info
            info_fields = {
                "Current Price": info.get("currentPrice"),
                "Trailing Eps": info.get("trailingEps"),
                "Book Value": info.get("bookValue"),
                "Market Cap": info.get("marketCap"),
                "Enterprise Value": info.get("enterpriseValue"),
                "Dividend Yield": info.get("dividendYield"),
                "Shares Outstanding": info.get("sharesOutstanding"),
                "Earnings Growth": info.get("earningsGrowth"),
            }

            for k, v in info_fields.items():
                df[k] = v

            df["Symbol"] = ticker
            df = df.reset_index().rename(columns={"index": "Date"})
            
            cols = ["Date", "Symbol"] + [c for c in df.columns if c not in ["Date", "Symbol"]]
            df = df[cols]

            all_data.append(df)
            print(f"[{i}/{len(tickers)}] Collected variables for {ticker}")

        except Exception as e:
            print(f"[{i}/{len(tickers)}] Error fetching variables for {ticker}: {e}")

    variables = pd.concat(all_data, ignore_index=True)
    return variables

variables = calculate_variables(tickers)

[1/503] Collected variables for A
[2/503] Collected variables for AAPL
[3/503] Collected variables for ABBV
[4/503] Collected variables for ABNB
[5/503] Collected variables for ABT
[6/503] Collected variables for ACGL
[7/503] Collected variables for ACN
[8/503] Collected variables for ADBE
[9/503] Collected variables for ADI
[10/503] Collected variables for ADM
[11/503] Collected variables for ADP
[12/503] Collected variables for ADSK
[13/503] Collected variables for AEE
[14/503] Collected variables for AEP
[15/503] Collected variables for AES
[16/503] Collected variables for AFL
[17/503] Collected variables for AIG
[18/503] Collected variables for AIZ
[19/503] Collected variables for AJG
[20/503] Collected variables for AKAM
[21/503] Collected variables for ALB
[22/503] Collected variables for ALGN
[23/503] Collected variables for ALL
[24/503] Collected variables for ALLE
[25/503] Collected variables for AMAT
[26/503] Collected variables for AMCR
[27/503] Collected variables for AMD
[

/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/3983459700.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  variables = pd.concat(all_data, ignore_index=True)


In [4]:
#DATE

print("Min Date:", variables["Date"].min())
print("Max Date:", variables["Date"].max())
variables = variables.drop(columns=["Date"])

Min Date: 2025-05-31 00:00:00
Max Date: 2025-08-31 00:00:00


In [5]:
#METRICS

def calculate_metrics(df):
    #valuation
    df["PER"] = df["Current Price"] / df["Trailing Eps"]
    df["PEGR"] = df["PER"] / (df["Earnings Growth"] * 100)
    df["PBR"] = df["Current Price"] / df["Book Value"]
    df["DY"] = df["Dividend Yield"]
    df["EV/EBITDA"] = df["Enterprise Value"] / df["EBITDA"]
    df["P/S"] = df["Current Price"] * df["Shares Outstanding"] / df["Total Revenue"]

    #profitability
    df["ROA"] = df["Net Income"] / df["Total Assets"]
    df["NPM"] = df["Net Income"] / df["Total Revenue"]

    #financial health
    df["ICR"] = df["EBIT"] / df["Interest Expense"]
    df["FCF"] = df["Free Cash Flow"]

    df.replace([float("inf"), -float("inf")], pd.NA, inplace=True)

    cols_to_remove = [
        'Total Assets', 'Inventory', 'Total Debt', 'Net Income', 'EBITDA', 'EBIT',
        'Operating Income', 'Gross Profit', 'Total Revenue', 'Interest Expense',
        'Free Cash Flow', 'Current Price', 'Trailing Eps', 'Book Value', 'Market Cap', 
        'Enterprise Value', 'Dividend Yield', 'Shares Outstanding', 
        'Earnings Growth'
    ]
    df = df.drop(columns=[c for c in cols_to_remove if c in df.columns])

    return df

metrics = calculate_metrics(variables)

/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/2251570529.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace([float("inf"), -float("inf")], pd.NA, inplace=True)


In [6]:
#VARIABLES 2

def calculate_variables_2(tickers):
    all_data = []

    for i, ticker in enumerate(tickers, 1):
        try:
            t = yf.Ticker(ticker)

            is_ = t.quarterly_income_stmt.T
            is_cols = ['Net Income', 'Total Revenue']
            is_ = is_[[c for c in is_cols if c in is_.columns]]

            df = is_

            df["Symbol"] = ticker
            df = df.reset_index().rename(columns={"index": "Date"})
            
            cols = ["Date", "Symbol"] + [c for c in df.columns if c not in ["Date", "Symbol"]]
            df = df[cols]

            all_data.append(df)
            print(f"[{i}/{len(tickers)}] Collected variables for {ticker}")

        except Exception as e:
            print(f"[{i}/{len(tickers)}] Error fetching variables for {ticker}: {e}")

    variables2 = pd.concat(all_data, ignore_index=True)
    return variables2

variables2 = calculate_variables_2(tickers)

[1/503] Collected variables for A
[2/503] Collected variables for AAPL
[3/503] Collected variables for ABBV
[4/503] Collected variables for ABNB
[5/503] Collected variables for ABT
[6/503] Collected variables for ACGL
[7/503] Collected variables for ACN
[8/503] Collected variables for ADBE
[9/503] Collected variables for ADI
[10/503] Collected variables for ADM
[11/503] Collected variables for ADP
[12/503] Collected variables for ADSK
[13/503] Collected variables for AEE
[14/503] Collected variables for AEP
[15/503] Collected variables for AES
[16/503] Collected variables for AFL
[17/503] Collected variables for AIG
[18/503] Collected variables for AIZ
[19/503] Collected variables for AJG
[20/503] Collected variables for AKAM
[21/503] Collected variables for ALB
[22/503] Collected variables for ALGN
[23/503] Collected variables for ALL
[24/503] Collected variables for ALLE
[25/503] Collected variables for AMAT
[26/503] Collected variables for AMCR
[27/503] Collected variables for AMD
[

In [7]:
#DATE 2

variables2 = variables2.drop(columns=["Date"])

In [8]:
#METRICS 2

def calculate_metrics_2(df):
    #growth
    df["RGR"] = df.groupby("Symbol")["Total Revenue"].pct_change()
    df["EGR"] = df.groupby("Symbol")["Net Income"].pct_change()

    df.replace([float("inf"), -float("inf")], pd.NA, inplace=True)

    cols_to_remove = ['Total Revenue', 'Net Income']
    df = df.drop(columns=[c for c in cols_to_remove if c in df.columns])

    return df

metrics2 = calculate_metrics_2(variables2)

/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/3141918789.py:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["RGR"] = df.groupby("Symbol")["Total Revenue"].pct_change()
/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/3141918789.py:6: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["EGR"] = df.groupby("Symbol")["Net Income"].pct_change()


In [9]:
#MERGE

def last_nonzero(df, group_col, value_cols):
    result = []
    for symbol, group in df.groupby(group_col):
        group = group.sort_index()
        non_zero_mask = (group[value_cols] != 0).any(axis=1)
        non_zero_rows = group[non_zero_mask]
        if not non_zero_rows.empty:
            result.append(non_zero_rows.tail(1))
        else:
            result.append(group.tail(1))
            
    return pd.concat(result).reset_index(drop=True)

value_cols = ["RGR", "EGR"] 
all_metrics = pd.merge(metrics, metrics2, on="Symbol", how="inner")
all_metrics = last_nonzero(all_metrics, "Symbol", value_cols)
all_metrics = all_metrics.groupby("Symbol").tail(1).reset_index(drop=True)

In [10]:
#MISSING VALUES

missing_per_symbol = all_metrics.isna().groupby(all_metrics["Symbol"]).sum().sum(axis=1)
missing_per_symbol = missing_per_symbol[missing_per_symbol > 0]

print("Missing Values per Symbol:")
for symbol, count in missing_per_symbol.sort_values(ascending=False).items():
    print(f"- {symbol}: {count}")

missing_per_column = all_metrics.isna().sum()
missing_per_column = missing_per_column[missing_per_column > 0]

print("\nMissing Values per Column:")
for col, count in missing_per_column.sort_values(ascending=False).items():
    print(f"- {col}: {count}")

Missing Values per Symbol:
- AZO: 7
- JBL: 6
- MU: 6
- FDS: 6
- CTAS: 6
- COST: 6
- ACN: 6
- LEN: 6
- TTWO: 3
- COF: 3
- DASH: 3
- RF: 3
- CRWD: 2
- EPAM: 2
- PNC: 2
- EW: 2
- ERIE: 2
- RJF: 2
- PRU: 2
- CPRT: 2
- SCHW: 2
- DAY: 2
- SYF: 2
- CSGP: 2
- STT: 2
- SOLV: 2
- FDX: 2
- INTC: 2
- MNST: 2
- KEY: 2
- JPM: 2
- ISRG: 2
- MRNA: 2
- MS: 2
- MTB: 2
- INCY: 2
- FFIV: 2
- ABNB: 2
- HOOD: 2
- HBAN: 2
- GS: 2
- NTRS: 2
- FITB: 2
- PFG: 2
- PLTR: 2
- LULU: 2
- CNC: 2
- TTD: 2
- WDC: 2
- C: 2
- ALGN: 2
- WBD: 2
- BRK-B: 2
- BK: 2
- BAC: 2
- AIG: 2
- BA: 2
- AXP: 2
- VRTX: 2
- USB: 2
- ANET: 2
- ULTA: 2
- WFC: 2
- BX: 2
- XYZ: 2
- CBRE: 2
- ACGL: 2
- CMG: 2
- ADSK: 2
- CFG: 2
- TDY: 2
- TFC: 2
- MPWR: 1
- NVR: 1
- NRG: 1
- MET: 1
- NOW: 1
- NKE: 1
- MGM: 1
- MHK: 1
- NFLX: 1
- MOS: 1
- VTRS: 1
- MOH: 1
- NCLH: 1
- WAT: 1
- WSM: 1
- MTD: 1
- NWSA: 1
- WRB: 1
- WDAY: 1
- NWS: 1
- UAL: 1
- ON: 1
- ORLY: 1
- TAP: 1
- SYK: 1
- TECH: 1
- SWK: 1
- SW: 1
- TPL: 1
- TPR: 1
- TRMB: 1
- STZ: 1
- MCHP:

In [11]:
#HANDLE MISSING VALUES

#stocks
symbols_to_drop = ["AZO", "JBL", "ACN", "LEN", "COST", "FDS", "MU", "CTAS"]
all_metrics = all_metrics[~all_metrics["Symbol"].isin(symbols_to_drop)].reset_index(drop=True)

#variables
sp500["Symbol"] = sp500["Symbol"].replace(ticker_map)
sp500 = sp500[["Symbol", "GICS Sector"]]
all_metrics = all_metrics.merge(sp500, on="Symbol", how="left")

all_metrics['DY'] = all_metrics['DY'].fillna(0)

all_metrics['ICR'] = all_metrics.groupby('GICS Sector')['ICR'].transform(
    lambda x: x.fillna(x.median())
)
all_metrics['ICR'] = all_metrics['ICR'].fillna(all_metrics['ICR'].median())

all_metrics['EV/EBITDA'] = all_metrics.groupby('GICS Sector')['EV/EBITDA'].transform(
    lambda x: x.fillna(x.median())
)
all_metrics['EV/EBITDA'] = all_metrics['EV/EBITDA'].fillna(all_metrics['EV/EBITDA'].median())

all_metrics['PEGR'] = all_metrics.groupby('GICS Sector')['PEGR'].transform(
    lambda x: x.fillna(x.median())
)
all_metrics['PEGR'] = all_metrics['PEGR'].fillna(all_metrics['PEGR'].median())

all_metrics = all_metrics.drop(columns=['GICS Sector'])

/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/4135206331.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.median())
/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/4135206331.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.median())
/var/folders/9b/b9bk135x3gd6hxg4hr0846t80000gn/T/ipykernel_21608/4135206331.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=F

In [12]:
#SAVE

all_metrics.to_csv("../Data/fundamental.csv", index=False)